swp continuum

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[2]:


import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import math
from math import log
from scipy import optimize
from scipy.interpolate import interp1d
from lmfit import Model, Parameters
from scipy.interpolate import UnivariateSpline
from math import exp, pi
from scipy.integrate import quad
import os

#  ***************************************

# (1) Import the data from txt file for MgII:

userinput=input ("Please enter the file you want to open")             # Asking the user to insert the file they want to fit
data= np.genfromtxt(userinput)                                              # Read data from the input file
x_Mg=data[216:523,0]
y_Mg=data[216:523,1]

# (2) Continuum windows required to determine the continuum level:
#The windows used here are approximately: (2301:2351 A) (2451:2502 A) (2602:2652 A) (2900:2950 A) (3050:3111 A)

x_continuum = np.concatenate([data[216:235,0],data[273:292,0],data[330:349,0],data[443:462,0],data[500:523,0]])
y_continuum = np.concatenate([data[216:235,1],data[273:292,1],data[330:349,1],data[443:462,1],data[500:523,1]])



# In[5]:


# (3) Define the power law to fit the continuum below the emission line:
def pl_f(x,amplitude,index):                                              #Define the power_law function of the form y=ax^b
    ret = amplitude*x**index
    return ret

# (3-1) Create model:
pl_model = Model(pl_f)

# (3-2) Create parameters:
param_pl = pl_model.make_params()                                 # Set initial values, min, max  of amplitude and index

param_pl.add('amplitude',value=1e-09)
param_pl.add('index',value=-0.7)


result_pl = pl_model.fit(y_continuum, param_pl,x=x_continuum)
print(result_pl.fit_report(min_correl=0.1))                                 #print the report

amplitude=result_pl.params['amplitude'].value                       #to save the amplitude value as "amplitude"
spectral_index=result_pl.params['index'].value                      #to save the index value as "spectral index"

amplitude_err=result_pl.params['amplitude'].stderr                  #to save error of amplitude
spectral_index_err=result_pl.params['index'].stderr                 #to save error of index
#amplitude,amplitude_err,spectral_index,spectral_index_err

# (4) Subtraction of the continuum:
Continuum_subtracted_spectrum=y_Mg-(amplitude*x_Mg**spectral_index)

# (5) Plotting the spectrum before and after continuum subtraction:
fig=plt.figure(figsize=(7,5))
plt.xlabel(r'Wavelength $(\rm \AA)$',fontsize=14)
plt.ylabel(r'Flux (erg cm$^{-2}$ S$^{-1}$ $\rm\AA^{-1})$',fontsize=14)
plt.plot(x_Mg, y_Mg, color='blue', label='Before continuum subtraction')
plt.plot(x_Mg, amplitude*x_Mg**spectral_index, color='green', label='Continuum level')                            #plot the poly
plt.scatter(x_continuum, y_continuum,color='red', label="Continuum windows") #plor the continuum windows
plt.plot(x_Mg, Continuum_subtracted_spectrum, color='orange', label='After continuum subtraction')
plt.axhline(y=0, linestyle='-')                   #plotting horizontal axis at 0
#plt.text(1800,1e-11,"{}".format(userinput_2))
plt.title('{}'.format(source_name))
plt.legend()
plt.savefig('{}_continuum subtration.png'.format(source_name))                              # Save the plot


#############################################################################################
####**********************************SNR********************************************



#finding the snr, Favg, Frms:

# for continuum window (2301:2351 A)
W1_best_fit=amplitude*(data[216:235,0])**(spectral_index)
Favg1=np.mean(W1_best_fit)
Frms1=math.sqrt(sum((W1_best_fit-data[216:235,1])**2)/W1_best_fit.shape)
snr1=Favg1/Frms1

# for continuum window (2451:2502 A)
W2_best_fit=amplitude*(data[273:292,0])**(spectral_index)
Favg2=np.mean(W2_best_fit)
Frms2=math.sqrt(sum((W2_best_fit-data[273:292,1])**2)/W2_best_fit.shape)
snr2=Favg2/Frms2

# for continuum window (2602:2652 A)
W3_best_fit=amplitude*(data[330:349,0])**(spectral_index)
Favg3=np.mean(W3_best_fit)
Frms3=math.sqrt(sum((W3_best_fit-data[330:349,1])**2)/W3_best_fit.shape)
snr3=Favg3/Frms3

# for continuum window (2900:2950 A)
W4_best_fit=amplitude*(data[443:462,0])**(spectral_index)
Favg4=np.mean(W4_best_fit)
Frms4=math.sqrt(sum((W4_best_fit-data[474:493,1])**2)/W4_best_fit.shape)
snr4=Favg4/Frms4

# for continuum window (3050:3111 A)
W5_best_fit=amplitude*(data[500:523,0])**(spectral_index)
Favg5=np.mean(W5_best_fit)
Frms5=math.sqrt(sum((W5_best_fit-data[500:523,1])**2)/W5_best_fit.shape)
snr5=Favg5/Frms5

print("************************************************")
print("---SNR:---")
print("Window           Favg                     Frms                     SNR")
print("2301:2351        {:g}                      {:g}                     {:g}".format(Favg1, Frms1, snr1))
print("2451:2502        {:g}                      {:g}                     {:g}".format(Favg2, Frms2, snr2))
print("2602:2652        {:g}                      {:g}                     {:g}".format(Favg3, Frms3, snr3))
print("2900:2950        {:g}                      {:g}                     {:g}".format(Favg4, Frms4, snr4))
print("3050:3111        {:g}                      {:g}                     {:g}".format(Favg5, Frms5, snr5))






from csv import writer
LWP_Fitting_results=[source_name,amplitude,amplitude_err, spectral_index,spectral_index_err,Favg1, Frms1, snr1,Favg2, Frms2, snr2,Favg3, Frms3, snr3,Favg4, Frms4, snr4, Favg5,Frms5,snr5]
with open('lwp-continuum fux powerlaw fit.csv', 'a') as f_object:
    writer_object = writer(f_object,lineterminator='\n')
    writer_object.writerow(LWP_Fitting_results)
    f_object.close()


# In[ ]:
